In [26]:
import pandas as pd
import numpy as np
import requests
import datetime

In [27]:
# This will be the function that pulls the rolling data. Will pull the past 8 months
def pull_data(df):
    today = datetime.datetime.now()
    time_format = '%Y-%m-%dT%H:%M:%S.%fZ'

    for i in range(4, 252, 3):
        date_end = datetime.date.today()
        delta_end = datetime.timedelta(days=i)
        delta_start = datetime.timedelta(days=i+3)
        date_end = date_end - delta_end
        date_start = today - delta_start

        date_end = date_end.strftime("%Y-%m-%d")
        date_start = date_start.strftime("%Y-%m-%d")
        response = requests.get(f"http://144.39.204.242:11236/evr/leviton/evr?dateStart={date_start}&dateEnd={date_end}")
        usage = response.json()

        ten_minute_averages = {"timestamp": [], "power": [], "isDay": [], "hour": []}

        for i in range(0, len(usage['data']), 85):
            values = usage['data'][i:i+120]
            power = [value['power'] for value in values]
            ten_minute_averages["timestamp"].append(values[0]['timestamp'])
            ten_minute_averages["power"].append(np.average(power) * 1000)
            test_date = datetime.datetime.strptime(values[0]['timestamp'], time_format)
            test_date = test_date - datetime.timedelta(hours=6)
            isDay =  1 if test_date.hour >= 7 and test_date.hour <= 19 else 0 # From 7:00 AM to 7 PM is day
            hour = test_date.hour
            ten_minute_averages["isDay"].append(isDay)
            ten_minute_averages["hour"].append(hour)
        
        temp_df = pd.DataFrame(ten_minute_averages)
        df = pd.concat([df, temp_df], ignore_index=True)
    
    
    return df

In [28]:
df = pd.DataFrame(columns=["timestamp", "power", "isDay", "hour"])
df= pull_data(df)
df.tail()
df.to_csv("power_usage.csv")

In [29]:
# time_format = '%Y-%m-%dT%H:%M:%S.%fZ'
# test = datetime.datetime.strptime('2023-03-22T23:59:52.878Z', time_format)